In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [3]:
from src.fn17 import load_dataset_hf, load_dataset_nltk
from src.train import train, test
from src.predict import predict
from src.process_data import prepare_data

2024-02-13 11:39:54.156426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 11:39:54.400899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 11:39:55.656304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
CHECKPOINT = 'bert-base-cased'
TASK = 'frames' # 'triggers' or 'frames'
dataset = load_dataset_nltk()

In [ ]:
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

## Train

In [18]:
OUT_DIR = f'./models/{TASK}'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [ ]:
train(pretrained_model=CHECKPOINT, task=TASK, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

## Test

In [ ]:
test(OUT_DIR, tokenized_dataset['test'], task=TASK)

## Predict

In [4]:
CHECKPOINT = './models/models/frames'
CHECKPOINT_TARGETS = './models/models/targets'

sentence = "When the moon hits your eye, that's 'amore'."
sentence2 = "The hallway smelt of boiled cabbage and old rag mats."

predict(pretrained_model=CHECKPOINT_TARGETS, sentence=sentence2, task='targets', visualize=True)
predict(pretrained_model=CHECKPOINT, sentence=sentence2, task='frames', visualize=True)